In [ ]:
!pip install transformers datasets wandb rouge_score accelerate pytorch-lightning peft tensorboardX
!pip install accelerate
# %pip install -U PyPDF2
# %pip install python-docx
# %pip install -U tokenizers
# %pip install accelerate -U
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 

In [ ]:
import pandas as pd
import numpy as np
import re
# from PyPDF2 import PdfReader
import os
# import docx
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

In [ ]:
#Functions to read different file types
# def read_pdf(file_path):
#     with open(file_path, "rb") as file:
#         pdf_reader = PdfReader(file)
#         text = ""
#         for page_num in range(len(pdf_reader.pages)):
#             text += pdf_reader.pages[page_num].extract_text()
#     return text

# def read_word(file_path):
#     doc = docx.Document(file_path)
#     text = ""
#     for paragraph in doc.paragraphs:
#         text += paragraph.text + "\n"
#     return text

# def read_txt(file_path):
#     with open(file_path, "r") as file:
#         text = file.read()
#     return text

# def read_documents_from_directory(directory):
#     combined_text = ""
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         if filename.endswith(".pdf"):
#             combined_text += read_pdf(file_path)
#         elif filename.endswith(".docx"):
#             combined_text += read_word(file_path)
#         elif filename.endswith(".txt"):
#             combined_text += read_txt(file_path)
#     return combined_text

In [ ]:
# train_directory = '/content/drive/MyDrive/ColabData/nlpTermproject'
# text_data = read_documents_from_directory(train_directory)
# text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [ ]:
# with open(/content/Colabdata/nlptermproject/train.txt, "w") as f:
#     f.write(text_data)

In [ ]:

def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = GPT2LMHeadModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
        logging_steps=300,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()

train_file_path = "/content/drive/MyDrive/ColabData/nlpTermproject/Train_v2.txt"
model_name = 'skt/kogpt2-base-v2'
output_dir = '/content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 300.0
save_steps = 50000



In [ ]:
# #LORA
# def load_dataset(file_path, tokenizer, block_size=128):
#     dataset = TextDataset(
#         tokenizer=tokenizer,
#         file_path=file_path,
#         block_size=block_size,
#     )
#     return dataset

# def load_data_collator(tokenizer, mlm=False):
#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer,
#         mlm=mlm,
#     )
#     return data_collator

# def train(train_file_path, model_name, output_dir, overwrite_output_dir,
#           per_device_train_batch_size, num_train_epochs, save_steps):
#     tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
#     train_dataset = load_dataset(train_file_path, tokenizer)
#     data_collator = load_data_collator(tokenizer)

#     tokenizer.save_pretrained(output_dir)

#     model = GPT2LMHeadModel.from_pretrained(model_name)

#     # LoRA 설정
#     peft_config = LoraConfig(
#         task_type=TaskType.CAUSAL_LM,  # causal language modeling task
#         inference_mode=False,  # 학습 모드
#         r=8,  # LoRA의 rank
#         lora_alpha=32,  # LoRA의 alpha
#         lora_dropout=0.1,  # LoRA dropout
#     )

#     model = get_peft_model(model, peft_config)

#     model.save_pretrained(output_dir)

#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         overwrite_output_dir=overwrite_output_dir,
#         per_device_train_batch_size=per_device_train_batch_size,
#         num_train_epochs=num_train_epochs,
#         save_steps=save_steps,
#         logging_steps=300,  # 로그 출력 주기 설정
#         logging_dir='./logs',  # 로그 디렉토리 설정
#         log_level='info',  # 로그 레벨 설정
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         data_collator=data_collator,
#         train_dataset=train_dataset,
#     )

#     trainer.train()
#     trainer.save_model()

# train_file_path = "/content/drive/MyDrive/ColabData/nlpTermproject/Train_mini.txt"
# model_name = 'skt/kogpt2-base-v2'
# output_dir = '/content/drive/MyDrive/ColabData/nlpTermproject/output6_mini_50epoch'
# overwrite_output_dir = True
# per_device_train_batch_size = 8
# num_train_epochs = 50.0
# save_steps = 50000

In [ ]:

# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon":

Step,Training Loss
300,0.609400
600,0.022700
900,0.009700
1200,0.005900
1500,0.005600
1800,0.004000
2100,0.005300
2400,0.003200
2700,0.002500
3000,0.002400




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch
Configuration saved in /content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch/config.json
Configuration saved in /content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch/generation_config.json
Model weights saved in /content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch/model.safetensors


In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:

model_path = '/content/drive/MyDrive/ColabData/nlpTermproject/output6_full_300epoch'
sequence = "[Q]스파이더맨의 줄거리를 알려줘"
max_length = 80

generate_text(model_path, sequence, max_length)

[Q]스파이더맨의 줄거리를 알려줘
[A]영화 제목: 스파이더맨
줄거리: 피터 파커의 정체가 드러난 후, 그는 자신의 삶을 되돌리기 위해 닥터 스트레인지에게 도움을 요청한다. 그러나 실험이 실패하면서 멀티버스가 열리고, 다른 차원의 적들이 나타난다. 피터는 자신의 정체성과 책임을 다시 한 번 고민하며, 새로운 도전에 맞
